# Simple Stacked Bar Chart


In [1]:
import altair as alt
import ipywidgets as widgets
from ipywidgets import interactive, interact, HBox, Layout, VBox, interact_manual
from IPython.display import display
import requests 
import pandas as pd
import json

### Fetch and examine data

In [2]:
# replace the url with the url to your data
url = "https://wri-rw.carto.com/api/v2/sql?q=SELECT data.incomegroup AS x, SUM(data.rural_population_data) AS rural_population, SUM(data.urban_population_data) AS urban_population FROM cli_022_populations_in_coastal_zones data WHERE EXTRACT(YEAR FROM datetime)=2010 GROUP BY data.incomegroup ORDER BY incomegroup!='Low income',incomegroup!='Lower middle income',incomegroup!='Upper middle income',incomegroup!='High income: OECD',incomegroup!='High income: nonOECD',incomegroup!='Not classified'"

In [3]:
# fetch and examine the data as a pandas dataframe 
r = requests.get(url)
df = pd.DataFrame(json.loads(r.content)['rows'])
df.head()

,rural_population,urban_population,x
0,140305790,63704774,Low income
1,544262407,368219316,Lower middle income
2,263456410,475137021,Upper middle income
3,107986538,483357900,High income: OECD
4,7309801,39251090,High income: nonOECD


In [4]:
# if the data looks good, create an altair.UrlData object from the url 
data = alt.UrlData(url, format = alt.JsonDataFormat(property='rows'))

### Visualize data in a stacked bar chart 

In [11]:
# define the function that creates the stacked bar chart 
def make_stacked_bar_chart(x_label, x_variable, x_datatype, x_zero, x_grid, x_format, x_transform, x_tooltip_label, x_tooltip_format, 
                            y_label, y_variable, y_datatype,  y_zero, y_grid, y_format, y_transform, y_tooltip_label, y_tooltip_format,
                           c_label, c_variable, c_datatype, c_tooltip_label, c_tooltip_format,
                          fold_transform):
    
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X("{}:{}".format(x_variable, x_datatype), axis=alt.Axis(title=x_label, grid=x_grid, format=x_format, labelAngle=-45), scale=alt.Scale(zero=x_zero)),
        y=alt.Y("{}:{}".format(y_variable, y_datatype), axis=alt.Axis(title=y_label, grid=y_grid, format=y_format), scale=alt.Scale(zero=y_zero)),
        color=alt.Color("{}:{}".format(c_variable, c_datatype), legend=alt.Legend(title=c_label)),
        tooltip=[alt.Tooltip("{}:{}".format(x_variable, x_datatype), title=x_tooltip_label, format=x_tooltip_format),
            alt.Tooltip("{}:{}".format(y_variable, y_datatype), title=y_tooltip_label, format=y_tooltip_format),
            alt.Tooltip("{}:{}".format(c_variable, c_datatype), title=c_tooltip_label, format=c_tooltip_format)]
           ).properties(
        width=800,
        height=500)
    
    if x_transform:
        chart = chart.transform_calculate(x=x_transform)
    
    if y_transform:
        chart = chart.transform_calculate(y=y_transform)
        
    if fold_transform != 'None': 
        chart = chart.transform_fold(fold_transform)
    
    display(chart)
    
    return chart

In [12]:
# create the chart using ipywidgets 
children = [widgets.Output() for _ in range(4)]
control = widgets.HBox(children[:-1]) 
display(control)
output = widgets.VBox([children[-1]])
display(output)

w = interactive(make_stacked_bar_chart,  
             {'manual': True},
             x_label = '', x_variable = 'x', x_datatype = ['N', 'O', 'T', 'Q', 'G'],  x_zero=False, 
                x_grid=False, x_format = '', x_transform = '', x_tooltip_label='Income Group', x_tooltip_format='',
             y_label = 'Coastal Population', y_variable ='value', y_datatype = ['Q', 'O', 'N', 'T', 'G'], y_zero=False,
                y_grid=True, y_format = '.1s', y_transform = '', y_tooltip_label='Population', y_tooltip_format='.2s',
             c_label = 'Segment', c_variable = 'key', c_datatype = ['N', 'O', 'Q', 'T', 'G'], c_tooltip_label='Segment', c_tooltip_format='',
               fold_transform=widgets.SelectMultiple(options = list(df) + [None]))
    
with children[0]:
    display(VBox(w.children[:9]))
    
with children[1]:
    display(VBox(w.children[9:18]))
    
with children[2]:
    display(VBox(w.children[18:-1]))
    
with children[3]:
    display(w.children[-1])

# For data types, refer to the documentation at https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types
# The make_stacked_bar_chart function only includes transform_calculate and fold_transform for now
# To add more transformations, refer to the documentation at https://altair-viz.github.io/user_guide/transform/index.html
# After filling in all the parameters, click the 'Run Interact' button. An empty chart indicates an error has occurred. 
# In that case, please double check if x_variable, y_variable, and c_variable have been set to the correct values. 